## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([128]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([128, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])  # 展平输入图像
        hidden = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(hidden, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 112.491936 ; accuracy 0.096
epoch 1 : loss 100.71522 ; accuracy 0.10498333
epoch 2 : loss 93.016045 ; accuracy 0.11195
epoch 3 : loss 87.22432 ; accuracy 0.117
epoch 4 : loss 82.46303 ; accuracy 0.12233333
epoch 5 : loss 78.405975 ; accuracy 0.1272
epoch 6 : loss 74.89165 ; accuracy 0.13166666
epoch 7 : loss 71.838776 ; accuracy 0.13766667
epoch 8 : loss 69.18585 ; accuracy 0.14371666
epoch 9 : loss 66.880264 ; accuracy 0.14961667
epoch 10 : loss 64.87141 ; accuracy 0.15561667
epoch 11 : loss 63.104225 ; accuracy 0.16133334
epoch 12 : loss 61.527348 ; accuracy 0.167
epoch 13 : loss 60.09998 ; accuracy 0.17215
epoch 14 : loss 58.791 ; accuracy 0.17691667
epoch 15 : loss 57.57522 ; accuracy 0.18131667
epoch 16 : loss 56.4326 ; accuracy 0.18541667
epoch 17 : loss 55.349083 ; accuracy 0.18976666
epoch 18 : loss 54.31509 ; accuracy 0.19375
epoch 19 : loss 53.3222 ; accuracy 0.19723333
epoch 20 : loss 52.36444 ; accuracy 0.20068334
epoch 21 : loss 51.437557 ; accuracy 0.2042
e